### 0 Helper function

In [1]:
import yaml
import json


In [2]:
x = 10.1234567
x = float(f"{x:.1f}")
print(x)

10.1


In [37]:
class helperfunction():
    def load_file(self,filepath):
        with open(filepath,'r',encoding="utf-8") as f:
            return f.read()
    def load_yaml(self,filepath):
        with open(filepath,'r',encoding="utf-8") as f:
            return yaml.safe_load(f)
    def fop(self,float_num):
        return float(f"{float_num:.1f}")
    def jsonstr(self,ip):
        return str(json.dumps(ip,indent=4, ensure_ascii=False))



class A(helperfunction):
    def __init__(self):
        pass
    def fn(self):
        self.config = self.load_yaml("model.yaml")
        x = self.fop(10.12345)
        print(x)

a = A()
a.fn()

10.1


<hr>

### 01 Input CVResume

In [4]:
hp = helperfunction()
resume_json = hp.load_file("resume_json.txt")
print(resume_json)

{
  "contact_information": {
    "name": "Surya Teja Menta",
    "email": "-",
    "phone": "+91 8309584461",
    "linkedin": "-",
    "jobdb_link": "-",
    "portfolio_link": "suryatejamenta.co.in"
  },
  "professional_summary": {
    "has_summary": "Yes",
    "summary_points": [
      "I’m Surya Teja Menta, Results-driven Senior Data Scientist with 4+ years of experience in Data Science, Machine Learning (ML), and Generative AI (GenAI).",
      "Proven expertise in RAG (Retrieval-Augmented Generation), LLM fine-tuning, MLOps, and end-to-end AI solutions.",
      "Strong background in data analytics, statistical modeling, AI-powered automation, and scalable AI architectures.",
      "IBM Certified Professional Data Scientist with hands-on experience in LangChain, Hugging Face, OpenAI APIs, Vector Databases (ChromaDB, Pinecone), and cloud deployments (AWS, GCP).",
      "Passionate about AI research, model optimization, and developing cutting-edge AI solutions."
    ]
  },
  "education

<hr>

### 02 Promptbuilder

In [5]:
from google import genai
import json
import os
import yaml

In [6]:
class PromptBuilder(helperfunction):
    def __init__(self,section,criteria,cvresume):
        self.section  = section
        self.criteria = criteria[::-1]
        self.cvresume = cvresume
        self.config = self.load_yaml("prompt.yaml")
        # print(self.config)
    def build_response_template(self):
        return {
            "section": self.section,
            "scores": {
                c: {"score": 0, "feedback": ""} for c in self.criteria
            }
        }
    def build(self):
        config_role      = self.config['role']['role1']
        config_objective = self.config['objective']['objective1']
        config_section   = self.config['section']['section1']
        config_expected  = self.config['expected_content'][self.section]
        config_criteria  = ""
        for item in self.criteria:
            config_criteria = f"- {item}\n" + config_criteria
        config_scale     = self.config['scale']['score1']
        # config_output    = self.config['output']['format1']
        
        prompt_role      = f"Role :\n{config_role}"+"\n\n"
        prompt_objective = f"objectvie :\n{config_objective}"+"\n\n"
        prompt_section   = f"section :\n{config_section}"+"\n\n"
        prompt_expected  = f"expected :\n{config_expected}"+"\n"
        prompt_criteria  = f"Criteria :\n{config_criteria}"+"\n"
        prompt_scale     = f"Scale :\n{config_scale}"+"\n"
        prompt_output    = f"output :\n{json.dumps(self.build_response_template(), indent=2)}"+"\n\n"
        prompt_cvresume  = f"CV/Resume: \n{self.cvresume}"+"\n"

        prompt = prompt_role + prompt_objective + prompt_section \
            + prompt_expected + prompt_criteria + prompt_scale \
            + prompt_output + prompt_cvresume
        prompt = prompt.replace("<section_name>",self.section)
        return prompt

In [7]:
p1 = PromptBuilder( 
    section  = "Profile", 
    criteria = ["Completeness", "ContentQuality"],
    cvresume = resume_json
)
prompt1 = p1.build()
# print(prompt1)

In [8]:
p2 = PromptBuilder( 
    section  = "Summary", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt2 = p2.build()
# print(prompt2)

In [9]:
p3 = PromptBuilder( 
    section  = "Education", 
    criteria = ["Completeness","RoleRelevance"],
    cvresume = resume_json
)
prompt3 = p3.build()
# print(prompt3)

In [10]:
p4 = PromptBuilder( 
    section  = "Experience", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt4 = p4.build()
# print(prompt4)

In [11]:
p5 = PromptBuilder( 
    section  = "Activities", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length"],
    cvresume = resume_json
)
prompt5 = p5.build()
# print(prompt5)

In [12]:
p6 = PromptBuilder( 
    section  = "Skills", 
    criteria = ["Completeness","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt6 = p6.build()
# print(prompt6)

<hr>

### 03 LLMCaller

In [13]:
import re
import json

In [14]:
class LlmCaller(helperfunction):
    def __init__(self):
        GOOGLE_API_KEY = self.load_yaml("global.yaml")["setting"]["GOOGLE_API_KEY"]
        self.client    = genai.Client(api_key=GOOGLE_API_KEY)
        self.model     = self.load_yaml("model.yaml")["model"]["generation_model"]
    def parser(self,resp):
        text = resp.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        data = json.loads(text)
        return data
    def hit(self,prompt):
        self.prompt = prompt
        self.resp = self.client.models.generate_content(
            model    = self.model,
            contents = self.prompt
            )
        data = self.parser(self.resp)
        return data

In [15]:
op = LlmCaller()

In [16]:
op1 = op.hit(prompt1)
op2 = op.hit(prompt2)
op3 = op.hit(prompt3)
op4 = op.hit(prompt4)
op5 = op.hit(prompt5)
op6 = op.hit(prompt6)

In [17]:
op1

{'section': 'Profile',
 'scores': {'ContentQuality': {'score': 5,
   'feedback': "The professional summary serves as an excellent profile section. It effectively highlights the candidate's core competencies, experience, and specific technical expertise (e.g., RAG, LLM fine-tuning, MLOps, GenAI). The use of action-oriented language and quantifiable experience (4+ years) adds to its strength. It clearly articulates the candidate's value proposition and career focus."},
  'Completeness': {'score': 5,
   'feedback': "The section is complete, providing a strong professional identity ('Surya Teja Menta, Results-driven Senior Data Scientist'), clear positioning with specific expertise areas, and a defined career direction. It avoids unnecessary personal details and focuses purely on professional accomplishments and aspirations. All expected criteria for a 'Profile' section are met or exceeded by the professional summary provided."}}}

In [18]:
op2

{'section': 'Summary',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': 'The summary is highly relevant for a Senior Data Scientist or AI/ML Engineer role, clearly highlighting specialized experience and skills directly applicable to advanced data science and generative AI positions. It immediately positions the candidate for relevant opportunities.'},
  'Length': {'score': 2,
   'feedback': 'The summary is 5 sentences long, slightly exceeding the recommended 2-4 sentence length for optimal conciseness. While each sentence adds value, aiming for a tighter 3-4 sentences would make it even more impactful.'},
  'Grammar': {'score': 5,
   'feedback': 'The grammar, spelling, and punctuation are excellent throughout the summary, contributing to a professional and polished presentation.'},
  'ContentQuality': {'score': 4,
   'feedback': "The content clearly highlights specific technical skills (e.g., RAG, LLM fine-tuning, MLOps), domain expertise (Data Science, ML, Generative AI), and

In [19]:
op3

{'section': 'Education',
 'scores': {'RoleRelevance': {'score': 3,
   'feedback': "The Bachelor's Degree in Computer Science is highly relevant for a Senior Data Scientist role, providing a strong foundational background. However, the inclusion of HSC (High School Certificate) and SSC (Secondary School Certificate) for a candidate with 4+ years of experience is generally unnecessary and does not contribute to the professional relevance for a senior-level position. While the core relevant education is present, the section's overall relevance is slightly diluted by less pertinent details."},
  'Completeness': {'score': 4,
   'feedback': 'All required elements are present for each educational entry: institution name, degree & field of study, dates attended, and GPA/honors. The entries are thoroughly detailed. While for an experienced professional, including secondary and high school education might be considered excessive, each individual entry is complete according to the specified crite

In [20]:
op4

{'section': 'Experience',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': 'The roles (Senior Data Scientist, Subject Matter Expert (Data Analyst)) and the described responsibilities are highly relevant to advanced Data Science, Machine Learning, and Generative AI positions. The work showcases expertise in current and in-demand technologies like YOLOv8, RAG, LLM fine-tuning, MLOps, and NLP Transformers.'},
  'Length': {'score': 4,
   'feedback': "The section provides a good amount of detail for each role (5-6 bullet points) without being excessively long, striking an appropriate balance for the candidate's experience level (4+ years)."},
  'Grammar': {'score': 5,
   'feedback': 'The grammar is impeccable, with clear, concise, and professional language used throughout the experience descriptions. No grammatical errors or typos were detected.'},
  'ContentQuality': {'score': 3,
   'feedback': "The content effectively showcases a strong technical skillset and relevant project expe

In [21]:
op5

{'section': 'Activities',
 'scores': {'Length': {'score': 0,
   'feedback': "The 'Activities' section is entirely missing from the resume. There is no content to evaluate for length."},
  'Grammar': {'score': 0,
   'feedback': "The 'Activities' section is entirely missing from the resume. Therefore, there is no text to evaluate for grammar."},
  'ContentQuality': {'score': 0,
   'feedback': "The 'Activities' section is entirely missing. While the 'Education' section briefly mentions 'Participated in paper presentations and won prizes' and 'Participated in school dramas, sports, etc.', these are not detailed enough to constitute an 'Activities' section as per the criteria (competitions, hackathons, club activities, project descriptions with responsibilities and tools/tech)."},
  'Completeness': {'score': 0,
   'feedback': "The 'Activities' section is completely absent from the resume. There is no dedicated section or sufficient details regarding competitions, hackathons, club activities

In [22]:
op6

{'section': 'Skills',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': "The listed skills are exceptionally relevant and critical for a Senior Data Scientist role, especially one with a focus on Machine Learning, Deep Learning, Generative AI, MLOps, and cloud deployment. It demonstrates a strong alignment with the candidate's professional experience and summary."},
  'Length': {'score': 5,
   'feedback': "The length of the skills section is optimal. It provides a comprehensive and detailed overview of the candidate's capabilities without being overly verbose. The clear categorization further enhances readability and efficiency."},
  'Completeness': {'score': 5,
   'feedback': 'The skills section is remarkably complete. It comprehensively covers all expected categories: a vast array of technical skills (Python, SQL, ML, Cloud - GCP Cloud Run, Amazon SageMaker), relevant tools (Git, TensorFlow/PyTorch, Google Data Studio, Docker, Langchain), well-articulated soft skills, and prog

<hr>

### 04 SectionScoreAggregator

In [23]:
import copy
class SectionScoreAggregator(helperfunction):
    def __init__(self):
        self.config          = self.load_yaml("weight.yaml")         # config
    def aggregate(self,llm_output:dict):
        self.llm_output      = llm_output             # op
        self.section         = llm_output["section"]  # Get section
        self.section_weights = self.config["weights"][self.section]  # config["weights"][section_key][criteria]
        ddict = {}
        total = 0.0
        # Protect multiple mutation when we run more than one time
        scores_copy = copy.deepcopy(self.llm_output["scores"])
        for criteria, body in scores_copy.items():
            raw = body["score"]
            w   = self.section_weights[criteria]
            weighted = raw / 5 * w
            body["score"] = weighted
            ddict[criteria] = body
            total = total + weighted
        return {
            "section": self.section,
            "total_score":total,
            "scores":ddict
        }

In [24]:
agg = SectionScoreAggregator()

In [25]:
s1 = agg.aggregate(op1)
s1

{'section': 'Profile',
 'total_score': 20.0,
 'scores': {'ContentQuality': {'score': 10.0,
   'feedback': "The professional summary serves as an excellent profile section. It effectively highlights the candidate's core competencies, experience, and specific technical expertise (e.g., RAG, LLM fine-tuning, MLOps, GenAI). The use of action-oriented language and quantifiable experience (4+ years) adds to its strength. It clearly articulates the candidate's value proposition and career focus."},
  'Completeness': {'score': 10.0,
   'feedback': "The section is complete, providing a strong professional identity ('Surya Teja Menta, Results-driven Senior Data Scientist'), clear positioning with specific expertise areas, and a defined career direction. It avoids unnecessary personal details and focuses purely on professional accomplishments and aspirations. All expected criteria for a 'Profile' section are met or exceeded by the professional summary provided."}}}

In [26]:
s2 = agg.aggregate(op2)
s2

{'section': 'Summary',
 'total_score': 42.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': 'The summary is highly relevant for a Senior Data Scientist or AI/ML Engineer role, clearly highlighting specialized experience and skills directly applicable to advanced data science and generative AI positions. It immediately positions the candidate for relevant opportunities.'},
  'Length': {'score': 4.0,
   'feedback': 'The summary is 5 sentences long, slightly exceeding the recommended 2-4 sentence length for optimal conciseness. While each sentence adds value, aiming for a tighter 3-4 sentences would make it even more impactful.'},
  'Grammar': {'score': 10.0,
   'feedback': 'The grammar, spelling, and punctuation are excellent throughout the summary, contributing to a professional and polished presentation.'},
  'ContentQuality': {'score': 8.0,
   'feedback': "The content clearly highlights specific technical skills (e.g., RAG, LLM fine-tuning, MLOps), domain expertise (Data 

In [27]:
s3 = agg.aggregate(op3)
s3

{'section': 'Education',
 'total_score': 14.0,
 'scores': {'RoleRelevance': {'score': 6.0,
   'feedback': "The Bachelor's Degree in Computer Science is highly relevant for a Senior Data Scientist role, providing a strong foundational background. However, the inclusion of HSC (High School Certificate) and SSC (Secondary School Certificate) for a candidate with 4+ years of experience is generally unnecessary and does not contribute to the professional relevance for a senior-level position. While the core relevant education is present, the section's overall relevance is slightly diluted by less pertinent details."},
  'Completeness': {'score': 8.0,
   'feedback': 'All required elements are present for each educational entry: institution name, degree & field of study, dates attended, and GPA/honors. The entries are thoroughly detailed. While for an experienced professional, including secondary and high school education might be considered excessive, each individual entry is complete accord

In [28]:
s4 = agg.aggregate(op4)
s4

{'section': 'Experience',
 'total_score': 40.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': 'The roles (Senior Data Scientist, Subject Matter Expert (Data Analyst)) and the described responsibilities are highly relevant to advanced Data Science, Machine Learning, and Generative AI positions. The work showcases expertise in current and in-demand technologies like YOLOv8, RAG, LLM fine-tuning, MLOps, and NLP Transformers.'},
  'Length': {'score': 8.0,
   'feedback': "The section provides a good amount of detail for each role (5-6 bullet points) without being excessively long, striking an appropriate balance for the candidate's experience level (4+ years)."},
  'Grammar': {'score': 10.0,
   'feedback': 'The grammar is impeccable, with clear, concise, and professional language used throughout the experience descriptions. No grammatical errors or typos were detected.'},
  'ContentQuality': {'score': 6.0,
   'feedback': "The content effectively showcases a strong technical sk

In [29]:
s5 = agg.aggregate(op5)
s5

{'section': 'Activities',
 'total_score': 0.0,
 'scores': {'Length': {'score': 0.0,
   'feedback': "The 'Activities' section is entirely missing from the resume. There is no content to evaluate for length."},
  'Grammar': {'score': 0.0,
   'feedback': "The 'Activities' section is entirely missing from the resume. Therefore, there is no text to evaluate for grammar."},
  'ContentQuality': {'score': 0.0,
   'feedback': "The 'Activities' section is entirely missing. While the 'Education' section briefly mentions 'Participated in paper presentations and won prizes' and 'Participated in school dramas, sports, etc.', these are not detailed enough to constitute an 'Activities' section as per the criteria (competitions, hackathons, club activities, project descriptions with responsibilities and tools/tech)."},
  'Completeness': {'score': 0.0,
   'feedback': "The 'Activities' section is completely absent from the resume. There is no dedicated section or sufficient details regarding competitions

In [30]:
s6 = agg.aggregate(op6)
s6

{'section': 'Skills',
 'total_score': 30.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': "The listed skills are exceptionally relevant and critical for a Senior Data Scientist role, especially one with a focus on Machine Learning, Deep Learning, Generative AI, MLOps, and cloud deployment. It demonstrates a strong alignment with the candidate's professional experience and summary."},
  'Length': {'score': 10.0,
   'feedback': "The length of the skills section is optimal. It provides a comprehensive and detailed overview of the candidate's capabilities without being overly verbose. The clear categorization further enhances readability and efficiency."},
  'Completeness': {'score': 10.0,
   'feedback': 'The skills section is remarkably complete. It comprehensively covers all expected categories: a vast array of technical skills (Python, SQL, ML, Cloud - GCP Cloud Run, Amazon SageMaker), relevant tools (Git, TensorFlow/PyTorch, Google Data Studio, Docker, Langchain), well-ar

<hr>

### 05 GlobalAggregator

In [31]:
from datetime import datetime,timezone,timedelta

In [32]:
class helperfunction():
    def load_file(self,filepath):
        with open(filepath,'r',encoding="utf-8") as f:
            return f.read()
    def load_yaml(self,filepath):
        with open(filepath,'r',encoding="utf-8") as f:
            return yaml.safe_load(f)
    def fop(self,float_num):
        return float(f"{float_num:.1f}")
    

class A(helperfunction):
    def __init__(self):
        pass
    def fn(self):
        self.config = self.load_yaml("model.yaml")
        x = self.fop(10.12345)
        print(x)

a = A()
a.fn()

10.1


In [46]:
class GlobalAggregator(helperfunction):
    def __init__(self,SectionScoreAggregator_output:list):
        self.section_outputs = SectionScoreAggregator_output
        self.timestamp       = str(datetime.now(tz=(timezone(timedelta(hours=7)))))
        self.model_config    = self.load_yaml("model.yaml")     # should include model name
        self.weight_config   = self.load_yaml("weight.yaml")    # includes weights + version
        self.prompt_config   = self.load_yaml("prompt.yaml")    # includes prompt version
    def fn1(self):
        weights = self.weight_config["weights"]
        contribution = {}
        total = 0.0
        for section_data in self.section_outputs:
            section_name    = section_data["section"]
            total_score     = section_data["total_score"]
            section_weight  = weights[section_name]["section_weight"]
            section_contrib = total_score * section_weight
            contribution[section_name] = {
                "section_total": total_score,
                "section_weight": section_weight,
                "contribution": self.fop(section_contrib)
            }
            total = total + section_contrib
        return {
            "final_resume_score":self.fop(total),
            "section_contribution":contribution
        }
    def fn2(self):
        details = {}
        for section_data in self.section_outputs:
            details[section_data["section"]] = {
                'total_score':section_data['total_score'],
                'scores':section_data['scores']
            }
        return details
    def fn3(self):
        return {
            "model_name": self.model_config['model']['generation_model'],
            "timestamp": self.timestamp,
            "weights_version": self.weight_config.get("version", "unknown"),
            "prompt_version": self.prompt_config.get("version", "unknown")
        }
    def fn0(self):
        conclution_part = self.fn1()
        detail_part     = self.fn2()
        metadata_part   = self.fn3()
        # return {
        #     "conclution":conclution_part,
        #     "section_detail":detail_part,
        #     "metadata":metadata_part
        # }
        return self.jsonstr({
            "conclution":conclution_part,
            "section_detail":detail_part,
            "metadata":metadata_part
        })

In [47]:
x = GlobalAggregator(
    SectionScoreAggregator_output = [s1,s2,s3,s4,s5,s6]
)

In [49]:
output = x.fn0()
print(output)

{
    "conclution": {
        "final_resume_score": 23.0,
        "section_contribution": {
            "Profile": {
                "section_total": 20.0,
                "section_weight": 0.1,
                "contribution": 2.0
            },
            "Summary": {
                "section_total": 42.0,
                "section_weight": 0.1,
                "contribution": 4.2
            },
            "Education": {
                "section_total": 14.0,
                "section_weight": 0.2,
                "contribution": 2.8
            },
            "Experience": {
                "section_total": 40.0,
                "section_weight": 0.2,
                "contribution": 8.0
            },
            "Activities": {
                "section_total": 0.0,
                "section_weight": 0.2,
                "contribution": 0.0
            },
            "Skills": {
                "section_total": 30.0,
                "section_weight": 0.2,
                "contribution

<hr>